# <font color=darkcyan>  A short introduction to machine Learning in Python </font>

### <font color=darkorange> Random forests, Gradient descent, Neural networks...</font>

## 1. <font color=darkred>  Random forests </font>

### <font color=darkred>  Random forests for time series prediction - simulated data </font>

Consider a Markov chain, given by $X_0 = x_0$ for $x_0\in \mathbb{R}^d$ and, for $k\geqslant 0$,
$$
X_{k+1} = \rho X_k + \sigma_X\varepsilon_k\,\,\, [2\pi]\,
$$ 
where $(\epsilon_k)_{k\geqslant 0}$ are i.i.d. standard Gaussian vectors in $\mathbb{R}^d$ $\sim {\sf N}(0,{\bf I}_d)$.

The observation model is
$$ 
Y_k = f(X_k) + \sigma_Y\eta_{k}\,,
$$
where $(\eta_k)_{k\geqslant 0}$ are i.i.d. $\sim N(0,1)$ and
$$
f:\begin{cases}
\mathbb{R}^d\to\mathbb{R}\\
{\bf x} \mapsto \sum_{i=1}^{d}\cos(x_i)
\end{cases}\,.
$$
The objective is to estimate the function $f$ using a training data set to predict the observations associated with the test dataset.

Start with a few imports, in particular the function ``RandomForestRegressor`` of sklearn that enables to apply the random forest algorithm in python.

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Define a function that takes as input the size $d$ of the vector $x_0$, the size $n$ of the dataset, the noise levels $\sigma_X$ of the autoregressive model and $\sigma_Y$ of the observed values and the parameter $\rho$ in the autoregressive model.

The function outputs data following the model described above.

In [ ]:
# function to sample a dataset 
def sample_data_ar(rho,sigmax,sigmay,n,d):
    X       = np.zeros(shape=(n,d))
    Y       = np.zeros(n)
    epsilon = np.random.normal(loc=0,scale=1,size = X.shape)
    eta     = np.random.normal(loc=0,scale=1,size = n)
    Y[0]    = np.sum(np.cos(X[0,:])) + sigmax*eta[0]
    for k in range(1,n):
        X[k,:] = (rho*X[k-1,:] + sigmax*epsilon[k,:])%(2*np.pi)
        Y[k]   = np.sum(np.cos(X[k,:])) + sigmay*eta[k]
    return X, Y

In [ ]:
# set parameters
n = 2000
d = 4

rho    = 0.1
sigmax = 0.1 
sigmay = 0.01 

Matrices X and Y containing data simulated according to the observation model described above may now be computed.

In this simulated toy model, the values of the hidden Markov chain $X_k$ are available in the matrix $X$. 
In practice, only the vector $Y$ is observed (hte values of $X_k$ must be estimated sequentially).

In [ ]:
# sample data  
X, Y = sample_data_ar(rho,sigmax,sigmay,n,d)

To evaluate different algorithms, the dataset is decomposed into training and test data.
In this type of time series analysis, the training data are the first values of $Y_k$ and the test are the last one. This corresponds to situations where one wants to predict future values of $Y$ given historical data.
This is somehow different form the i.i.d. case where taining and test data are chosen randomly using the function ``sklearn.model_selection.train_test_split``

In [ ]:
# split variables and observations, using 90% of the data set to estimate f 
import pandas as pd
df = pd.DataFrame(data = X, columns = ['X1', 'X2', 'X3', 'X4'])
df['Y'] = Y

nb_data_train = int(0.95*n)
nb_diff       = n-nb_data_train
df.head()
X_train = df.iloc[0:nb_data_train,:-1] 
X_test  = df.iloc[-nb_diff:,0:-1]
X_train.head()
Y_train = df.iloc[0:nb_data_train,-1] 
Y_test  = df.iloc[-nb_diff:,-1]

A first random forest prediction can be performed. 
``rf`` is the random forest function of sklearn when the number of trees in the forest is set to ``n\_trees``.
The forest is estimated using the training data ``X\_train`` and ``Y\_train`` and the values of $Y$ in the test set are compared to those predicted by the algorithm.
The mean-squared error between these quantities is also displayed.

In [ ]:
# number of trees in the forest for an elementary random forest estimate
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')

The predictions seem reasonable but it may be that these predictions are very accurate at first and then that the errors accumulate over time.
To test if this is the case, plot the values of $Y$ in the test set (true and predicted) as functions of the time from the last training time.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

To improve this first prediction, the different parameters of the random forest algorithm may be tuned. For example, an increase of the number of trees in the forest reduced the varaince of the Monte Carlo estimate.

In [ ]:
# number of trees in the forest
n_trees = 500
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)

plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')

Note that this increases substancially the computational time although predictions seem equally relevant.

In [ ]:
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

The objective is now to improve the algorithm using a cross-validation scheme to estimate the best value of these paramrters. It is useful to look at the parameters used by default in the algorithm here:
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Set a grid of parameters that will be tested by the cross-validation procedure.

In [ ]:
# with RandomizedSearchCVCreate, a grid of tuning parameters is built
# then a random search will be performed to test which parameter values yield the best 
# random forest estimate (see below).
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in the forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features is the number of dimension considered to select the best split (the dimension along which a cell is cut)
max_features = ['log2', 'sqrt']
# The maximum depth of the tree corresponds to the maximum number of levels of the tree. 
# If not given, splits are performed until all cells contain less than min_samples_split samples.
max_depth = np.arange(5,100,5)
# min_samples_split is the minimum number of samples in a cell to allow a split.
min_samples_split = [2, 3, 4, 5, 6, 7, 8]
# A split is considered in the tree if it leaves at least min_samples_leaf training samples in each 
# subcell obtained after the spliting process.
min_samples_leaf = [1, 2, 3, 4]
# if bootstrap is true all the training dataset is used to build each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

Exploring all parameters combinations in this grid would be computationally prohibitive. 
An efficient alternative is to compare parameters chosen at random in the grid.

In [ ]:
rf = RandomForestRegressor()
# Random search among  all parameters (the number of possible combination is given. It is not a complete grid search !!!). 
# search across n_iter = 100 different combinations with a default 3-fold cross validation.
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=10, n_jobs = -1)

rf_random.fit(X_train,Y_train)

In [ ]:
# display the best parameters
rf_random.best_params_

In [ ]:
# build the best explored random forest
best_random = rf_random.best_estimator_

In [ ]:
plt.figure(1,figsize=(8,8))
plt.title('Dimension of the state space: %d, MSE %f' %(d,mse))
plt.scatter(Y_test,y_pred_rf, color='b', s = 10, alpha = 0.8)
plt.plot(Y_test,Y_test,color='k', linestyle = 'dashed')

In [ ]:
# compute predictions usting test data and associated mean square error
y_pred_rf = rf_random.predict(X_test)
mse       = mean_squared_error(Y_test,y_pred_rf)
plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future values','Random forest estimates'))

### <font color=darkred> Random forests for time series prediction - Inflation prediction </font>

In [ ]:
# In this section, random forests are used to predic the Brazilian inflation based on
# many observed variables, see https://github.com/gabrielrvsc/HDeconometrics/
df = pd.read_csv('./BRinf')
df.head()

In [ ]:
# number of observations, number of variables
df.shape

The function ``pandas.DataFrame.corr`` may be used to compute the pairwise correlations between columns (variables and inflation). These correlations can be displayed using ``sns.heatmap`` to highlight highly correlated variables (and those likely to have an impact on the inflation).

See ``feature_importances_`` below for a first try at (relevant) variable selection.

In [ ]:
import seaborn as sns
corr = df.corr()
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(corr, xticklabels = False, yticklabels = False, cmap = 'Blues', ax = ax)

In [ ]:
corr.head()

In [ ]:
nb_data_train = 140
nb_diff       = df.shape[0]-nb_data_train
# inflation observations
Y_train = df.iloc[0:nb_data_train,1] 
Y_test  = df.iloc[-nb_diff:,1] 
Y_test.head()

In [ ]:
# other variables
X_train = df.iloc[0:nb_data_train,2:] 
X_test  = df.iloc[-nb_diff:,2:] 
X_test.head()

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test)

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

In [ ]:
# Many variables are used for the inflation prediction while very few observations are available.
# Selecting the most valuable variables is an alternativ to learn a simpler models
# This is obtained in Python with rf.feature_importances_

In [ ]:
plt.figure(1,figsize=(12,12))
plt.bar(list(df)[2:93],rf.feature_importances_,align='center')
plt.xticks(range(len(list(df)[2:93])),list(df)[2:93],rotation=90,size='small')
plt.title('Feature importances')

In [ ]:
# RandomForestClassifier and RandomForestRegressor use the gini importance mechanism as a measure of the 
# fetaures importance. The mean decrease in impurity importance of a 
# feature is computed by measuring t%%javascript
# see https://papers.nips.cc/paper/4928-understanding-variable-importances-in-forests-of-randomized-trees.pdf

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 10
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]

In [ ]:
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

<img src="./PCA_cor.png" width="500" height="700">

In [ ]:
# plot the MSE as a function of the numbers of variables used to estimate the function f
d_max = 60
MSE   = []
rf    = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train,Y_train)
for d in range(2,d_max): 
    rfd = RandomForestRegressor(n_estimators = n_trees)
    ind = rf.feature_importances_.argsort()[-d:]
    rfd.fit(X_train.iloc[:,ind],Y_train)
    # compute predictions usting test data and associated mean square error
    y_pred_rf = rfd.predict(X_test.iloc[:,ind])
    MSE = np.append(MSE,mean_squared_error(Y_test,y_pred_rf))

In [ ]:
#plt.figure(1,figsize=(8,8))
plt.plot(MSE,marker='o',color='k')
plt.xlabel('Number of features')
plt.ylabel('Mean square error on the test data set')

In [ ]:
# collect the indices of the features with highest importance.
nb_features_to_keep = 4
ind = rf.feature_importances_.argsort()[-nb_features_to_keep:]
X_train.iloc[:,ind]
n_trees = 100
rf      = RandomForestRegressor(n_estimators = n_trees)
rf.fit(X_train.iloc[:,ind],Y_train)

# compute predictions usting test data and associated mean square error
y_pred_rf = rf.predict(X_test.iloc[:,ind])

plt.figure(1,figsize=(8,8))
plt.plot(Y_test.values,marker='o',color='k')
plt.plot(y_pred_rf,'g--',marker='+')
plt.legend(labels=('Future inflation values','Random forest estimates'))

In [ ]:
# you may add here a cross validation procedure using RandomizedSearchCV or GridSearchCV
# to select the best parameters (and compare the mean square error with what you obtain above).

## 2. <font color=darkred> Gradient descent to estimate parameters </font>

In [ ]:
# Required packages
np.set_printoptions(precision=2) 
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import randn
from numpy.linalg import norm
from scipy.optimize import check_grad
import seaborn as sns

### <font color=darkred>  Losses and gradients </font>

The aim of this section is to detail how to solve the following optimization problem
$$
\arg\min_{w \in \mathbb R^d} \Big\{ f(w) + \frac{\lambda}{2} \|w\|_2^2 \Big\}\,,
$$
where $d$ is the number of features and where $f$ is $L$-smooth. Two classical settings:

**Linear regression**
$$
f: w \mapsto \frac 1n \sum_{i=1}^n f_i(w) = \frac{1}{2n} \sum_{i=1}^n (y_i - x_i' w)^2 + \frac{\lambda}{2} \|w\|_2^2 = \frac{1}{2 n} \| y - X w \|_2^2 + \frac{\lambda}{2} \|w\|_2^2\,,
$$
where $n$ is the sample size, $y = (y_1, \ldots, y_n)'\in\mathbb{R}^n$ and $X$ is the matrix of features with lines containing the features vectors $x_i \in \mathbb R^d$.

**Logistic regression**
$$
f: w \mapsto \frac 1n \sum_{i=1}^n f_i(w) = \frac{1}{n} \sum_{i=1}^n \log(1 + \exp(-y_i x_i' w)) + \frac{\lambda}{2} \|w\|_2^2\,,
$$
where $n$ is the sample size, and where $y_i \in \{ -1, 1 \}$ for all $1\leqslant i\leqslant n$.
 
Gradient descent algorithms require to compute $f(w)$, $\nabla f(w)$, $\nabla f_i(w)$ for stochastic gradient descent methods and $\partial_j \,f(w)$ for coordinate descent.

In [ ]:
def sigmoid(t):
    idx       = t > 0
    out       = np.empty(t.size)    
    out[idx]  = 1 / (1. + np.exp(-t[idx]))
    exp_t     = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out

def simu_logreg(w0, n_samples=1000, corr=0.5):
    # w0 : `numpy.array`, shape=(n_features,), parameters to be estimated
    # n_samples : `int`, default=1000, number of samples to simulate
    # corr : `float`, default=0.5, correlation of the features

    # Returns
    # X : `numpy.ndarray`, shape=(n_samples, n_features), simulated features matrix. 
    # Contains samples of a centered Gaussian vector with Toeplitz covariance.
    # y : `numpy.array`, shape=(n_samples,), simulated labels

    n_features = w0.shape[0]
    cov        = toeplitz(corr ** np.arange(0, n_features))
    X          = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    p          = sigmoid(X.dot(w0))
    y          = np.random.binomial(1, p, size=n_samples)
    y[:]       = 2 * y - 1
    return X, y

In [ ]:
n_samples = 500
w0        = np.array([-3, 3.])

X, y = simu_logreg(w0, n_samples=n_samples, corr = 0.8)

simulated_data          = pd.DataFrame(columns = ["x","y","Label"])
simulated_data["x"]     = X[:,0]
simulated_data["y"]     = X[:,1]
simulated_data["Label"] = y

sns.set_style("whitegrid")
sns.lmplot(x = "x", y = "y", data = simulated_data, fit_reg = False, hue = "Label", legend = True, scatter_kws={"s": 5})

plt.title("Logistic regression simulation")

In [ ]:
class ModelLogReg:
    # X : `numpy.array`, shape=(n_samples, n_features), features matrix
    # y : `numpy.array`, shape=(n_samples,), vector of labels
    # strength : `float`, strength of ridge penalization
    
    def __init__(self, X, y, strength):
        self.X = X
        self.y = y
        self.strength = strength
        self.n_samples, self.n_features = X.shape
    
    def loss(self, w):
        # Computes f(w)
        y, X, n_samples, strength = self.y, self.X, self.n_samples, self.strength
        res = 0
        for k in range(n_samples):
            res += np.log(1+np.exp(-y[k]*X[k].dot(w)))
        res = res/n_samples
        res += strength*0.5*(norm(w)**2)
        return res
        
       
    def grad(self, w):
        # Computes the gradient of f at w
        y, X, n_samples, strength = self.y, self.X, self.n_samples, self.strength
        res = 0
        for k in range(n_samples):
            res += (-y[k]*X[k]*np.exp(-y[k]*X[k].dot(w)))/(1+np.exp(-y[k]*X[k].dot(w)))
        res = res/n_samples
        res += strength*w
        return res
        

    def grad_i(self, i, w):
        # Computes the gradient of f_i at w
        X, strength = self.X, self.strength
        x_i = X[i]
        y   = self.y
        return (-y[i]*x_i*np.exp(-y[i]*x_i.dot(w)))/(1 + np.exp(-y[i]*x_i.dot(w))) + strength * w

    def lip(self):
        # Computes the Lipschitz constant of f
        X, n_samples = self.X, self.n_samples
        
        return((norm(X,2)**2)/(4*n_samples)+ self.strength)

In [ ]:
# Check numerically the gradient using the function checkgrad from scipy.optimize
# Use the function simu_logreg to simulate data according to the logistic regression model
n_features = 10
w_true     = np.random.randn(n_features)
X, y       = simu_logreg(w0 = w_true, n_samples=n_samples, corr=0.1)
modelreg   = ModelLogReg(X, y, strength=1e-3)
# check_grad assesses the correctness of a gradient by comparing it to a finite-difference approximation
check_grad(modelreg.loss, modelreg.grad, w_true)

#### <font color=darkred> Gradient descent </font>

Starting from an initial vector $w^{(0)}$ and with a step size $\eta>0$, the most simple gradient based algorithm writes, for all $k\geqslant 0$,
$$
w^{(k+1)} = w^{(k)} - \eta \nabla f(w^{(k)})\,.
$$
Assume that $f : \mathbb{R}^d \to \mathbb{R}$ is a $L$-smooth convex function, i.e. a convex function such that for all $(x,y)\in\mathbb{R}^d\times\mathbb{R}^d$, 
$$
\|\nabla f(x) - \nabla f(y)\|\leqslant L\|x-y\|
$$ 
and let $w^{\star}$ be a minimum of $f$ on $\mathbb{R}^d$. Then, Gradient Descent with step size $\eta \leqslant 1/L$ satisfies
$$
f(w^{(k)}) - f(w^{\star}) \leqslant \frac{\|w^{(0)} - w^{\star}\|_2^2}{2 \eta k}\,.
$$
In particular, for $\eta=1/L$, 
$$
L\|w^{(0)} - w^{\star}\|_2^2/2\varepsilon
$$
iterations are sufficient to get an $\varepsilon$-approximation of the minimal value of $f$.

In [ ]:
def gd(model, w0, n_iter):
    loss_val = np.zeros(n_iter+1)
    step     = 1 / model.lip()
    w        = w0.copy()
    w_new    = w0.copy()
    for k in range(n_iter + 1):
        w_new[:]    = w-step*model.grad(w)
        w[:]        = w_new
        loss_val[k] = model.loss(w)
    return loss_val

#### Choice of $\eta$

In practice, the algorithm performance is highly sensitive to the choice of $\eta$.

``Exact line search``. At each step, choose the best $\eta$ by optimizing
$$
\eta^{(k)} = \mathrm{argmin}_{\eta >0} f(w - \eta \nabla f(w))\,.
$$
The computational cost of this approach makes it prohibitive. 

``Backtracking line search``.
First, fix a parameter $0 < \beta < 1$, then at each iteration, start with $\eta^{(k)}=1$ and while
$$
f( w^{(k)} -  \eta^{(k)}\nabla f (w^{(k)})) > f(w^{(k)}) - \frac{\eta^{(k)}}{2} \|\nabla f(w^{(k)}) \|^2,
$$
update $ \eta^{(k)} = \beta \eta^{(k)}$. This simple to implement approach is widespread and efficient in practice. Let $f : \mathbb{R}^d \to \mathbb{R}$ be a $L$-smooth convex function and  $w^{\star}$ be a minimum of $f$ on $\mathbb{R}^d$. Then, Gradient Descent with backtracking line search satisfies
$$
f(w^{(k)}) - f(w^{\star}) \leqslant \frac{\|w^{(0)} - w^{\star}\|_2^2}{2 k \min(1, \beta/L) }\,,
$$
which provides a convergence result for a method with adaptive step size (i.e. it is not required to choose and tune $\eta$).

In the case of machine/deep learning applications, the function to be minimized is of the form:
$$
f:w\mapsto  \frac{1}{n}\sum_{i=1}^n \ell(Y_i, \langle w; X_i \rangle) + \lambda g(w) = \frac{1}{n}\sum_{i=1}^n f_i(w)\,.
$$
The previous method  is based on full gradients, since at each iteration  it requires to compute
$$
\nabla f(w) = \frac 1n \sum_{i=1}^n \nabla  f_i(w)\,,
$$
which depends on the whole dataset. When processing very large datasets ($n$ is large), this approach has a highly prohibitive computational cost  for a  unique step towards the minimum. Stochastic Gradient Descent algorithm aims at reducing this computational cost by replacing this gradient by an unbiased estimate. If $I$ choose uniformly at random $I \in \{ 1, \ldots, n \}$, then
$$
\mathbb{E}[\nabla f_I(w)] = \frac 1n \sum_{i=1}^n \nabla f_{i}(w) = \nabla f(w)\,.
$$
Therefore $\nabla f_I(w)$ is an \textbf{unbiased} estimate of the full gradient $\nabla f(w)$. The computation of $\nabla f_I(w)$ only requires the $I$-th data and its complexity is of order $O(d)$. Starting from $w^{(0)}$, and chosing the step size $\eta_k$ at each time step, Stochastic Gradient Descent algorithm proceeds as follows.
For all $k\geqslant 1$, pick at random (uniformly) $I_k$ in $\{ 1, \ldots, n \}$ and set
$$
w^{(k)} = w^{(k-1)} - \eta_k \nabla f_{I_k}(w^{(k-1)})\,.
$$
Each iteration has complexity $O(d)$ instead of $O(nd)$ for the previous  full gradient methods.

#### <font color=darkred> Stochastic gradient descent algorithms </font>

In [ ]:
def sgd(model, w0, n_iter, step):
    w         = w0.copy()
    n_samples = model.n_samples
    loss_val  = np.zeros(n_iter)
    for idx in range(n_iter):
        i             = np.random.randint(0, model.n_samples) 
        w             = w - step*model.grad_i(i,w)/(np.sqrt(idx+1))
        loss_val[idx] = model.loss(w)
    return loss_val

#### <font color=darkred> Accelerated gradient algorithm (bonus) </font>

In [ ]:
def agd(model, w0, n_iter):
    step  = 1 / model.lip()
    w     = w0.copy()
    w_new = w0.copy()
   
    # An extra variable is required for acceleration
    z         = w0.copy()
    t         = 1.
    t_new     = 1.    
    loss_val  = np.zeros(n_iter+1)
    for k in range(n_iter + 1):
        w_new[:]    = z-step*model.grad(w)
        t_new       = 1./2.+(1.+4.*t*t)**(0.5)/2.
        z[:]        = w_new+(t-1)/t_new*(w_new-w)
        t           = t_new
        w[:]        = w_new
        loss_val[k] = model.loss(w)
    return loss_val

In [ ]:
step_sgd   = 1e-1
w0         = np.random.randn(n_features)
n_iter_max = 10

gd_loss  = gd(modelreg, w0, n_iter = n_iter_max)
agd_loss = agd(modelreg, w0, n_iter = n_iter_max)
sgd_loss = sgd(modelreg, w0, n_iter = n_iter_max*n_samples, step = step_sgd)

In [ ]:
figure = plt.figure(figsize = (12, 8))
plt.subplot(1,3,1)
plt.plot(gd_loss, '-')
plt.title('Gradient descent')
plt.xlabel('Number of iterations')
plt.ylabel('Loss value')
plt.ylim([0.2,1])
plt.subplot(1,3,2)
plt.plot(agd_loss, '-')
plt.title('Accelerated gradient descent')
plt.xlabel('Number of iterations')
plt.ylabel('Loss value')
plt.ylim([0.2,1])
plt.subplot(1,3,3)
plt.plot(sgd_loss, ':')
plt.title('Stochastic gradient descent')
plt.xlabel('Number of iterations')
plt.ylim([0.2,1])
plt.tight_layout()

Repeat the experiment and highlight the influence of

``the number of observations`` ;

``the step-size`` (learning rate) of each algorithm ;

``the correlation`` between variables...

You may also repeat the process by defining a class for ``linear regression`` and computing the associated derivatives etc.

#### <font color=darkred> Optimization methods for neural networks (bonus) </font>

``The ADAptive GRADient`` algorithm introduced by http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf, (Duchi et al. 2011), starts from $w^{(0)}$ and uses a learning rate $\eta > 0$ and a momentum $\alpha$ and defines, for all $k\geqslant 0$ and all $j \in\{1, \ldots , d\}$, 
$$
w_j^{(k+1)} \gets w_j^{(k)}  - \frac{\eta}{\sqrt{\sum_{\tau=1}^k (\nabla f (w^{(\tau)}))_j^2}} (\nabla f(w^{(k)}))_j\,.
$$
The rationale of this method is that different rates are used for all coordinates which is crucial for neural networks in which gradient at different layers can be of different order of magnitude. It is proved in https://arxiv.org/pdf/1806.01811.pdf (Ward et al., 2018) that AdaGrad achieves the same convergence rate as gradient descent with optimal fixed stepsize up to a $\log$ factor. The adaptive step size grows with the inverse of the gradient magnitudes, so that large gradients have small learning rates and small gradients have
large learning rates.

``AdaDelta`` Introduced in https://arxiv.org/pdf/1212.5701.pdf (Zeiler, 2012) to reduce the sensitivity to initial conditions of AdaGrad. Indeed, if the initial gradients are large, the learning rates of AdaGrad will be low for all updates which can be overcome by increasing $\eta$, but making the AdaGrad method highly sensitive to the choice of $\eta$.  


``RMSprop optimizer`` Unpublished method, from the course of http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf (Geoff Hinton).


``ADAM: Adaptive moment estimation`` Introduced in https://arxiv.org/pdf/1412.6980.pdf (Kingma et al., 2014) and considered as the state of the art to otpimize neural networks, the ADAM procedure update the parameter estimate as follows. Starting from $m_0 = 0$ and $v_0=0$ and choosing $\beta_1, \beta_2, \eta, \varepsilon
 \in (0,1)$, compute first and second moment estimate 
$$
m_k = \beta_1 m_{k-1} + (1 - \beta_1) \nabla f (w^{(k)}) \quad  \mathrm{and} \quad v_k = \beta_2 v_{k-1} + (1 - \beta_2) (\nabla f (w^{(k)}))^2\,,
$$
then, compute the correction terms
$$
\hat{m}_k = \frac{m_k}{1 - \beta_1^k} \quad \hat{v}_k = \frac{v_k}{1 - \beta_2^k}\,,
$$
and update the parameter estimate with
$$
w^{(k+1)} = w^{(k)} - \frac{\eta}{\sqrt{\hat{v}_k}+ \varepsilon} \hat{m}_k\,.
$$
First convergence results can be found in https://arxiv.org/pdf/1412.6980.pdf (Kingma et al., 2014) and examples where ADAM algorithm does not converge to the optimum are given in https://openreview.net/pdf?id=ryQu7f-RZ (Reddi et al., 2018). Recent analysis by https://arxiv.org/abs/1810.02263 (Barakat et al., 2018).

## 3. <font color=darkred> Handwritten digit recognition with MNIST </font>

This section aims at using neural networks to perform classification on the [MNIST](http://yann.lecun.com/exdb/mnist) dataset.
This dataset contains images representing handwritten digits.  Each image is made of 28 x 28 pixels, and each pixel is represented by an integer (gray level).  These arrays can be flattened into vectors of 28 x 28 = 784 numbers.
Each image is then a 784-dimensional vector.  Visualisations of this vector space are given here: [http://colah.github.io/posts/2014-10-Visualizing-MNIST/](http://colah.github.io/posts/2014-10-Visualizing-MNIST/).

The labels in $\{0, 1, 2, \ldots, 9\}$ can represented using one-hot encoding: labels in $\{0, 1, 2, \ldots, 9\}$ are replaced by labels in $\{ 0, 1\}^{10}$, namely $0$ is replaced by $(1, 0, \ldots 0)$, $1$ is replaced by $(0, 1, 0, \ldots 0)$, $2$ is replaced by $(0, 0, 1, 0, \ldots, 0)$, etc.

Also, MNIST data is grayscale pixels in $\{0, \ldots, 255\}$ which should be normalized to belong to $[0, 1]$.

In [ ]:
from keras.datasets import mnist
#from keras import backend as K
# Number of classes
num_classes = 10
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train     = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test      = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(x_train[i].reshape(28, 28), 
               interpolation="none", cmap="gray_r")
    plt.title('Label = %d' % y_train[i], fontsize=14)
    plt.axis("off")
plt.tight_layout()

In [ ]:
n_rows = 4
n_cols = 8
plt.figure(figsize=(8, 4))
for i in range(n_rows * n_cols):
    plt.subplot(n_rows, n_cols, i+1)
    plt.imshow(x_train[i].reshape(28, 28),
               interpolation="none", cmap="gray_r")
    plt.axis("off")
plt.tight_layout()

In [ ]:
print(np.array2string(x_train[0].astype(np.int).reshape(28, 28),max_line_width=150))

In [ ]:
x_train /= 255
x_test /= 255
print(x_train.min(axis=None), x_train.max(axis=None))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import activations
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train[:10]

### <font color=darkred> Softmax regression </font>

The aim is to predict the digit $k$ represented by each picture, for $k \in \{0, \ldots, 9\}$.

Softmax regression provides a model for the probability that an input image $x$ is associated with each class using a simple linear model.  It is assumed that the probability to belong to the class $k$ can be expressed by a weigthed sum of the pixel intensities, with weights $W_{k, 1}, \ldots, W_{k, 784}$ and plus a bias $b_k$ capturing some variability independent of the input:
$$
\text{score}_k(x_i) = \sum_{j=1}^{784} W_{k, j} x_j + b_k,
$$
These scores are sometimes called the "logits" in the deep learning community.
The softmax function is then used to convert these scores into probabilities $p_k$:
$$
p_k(x_i) = \text{softmax}(\text{score}_k(x_i)) = \frac{\exp(\text{score}_k(x_i))}{\sum_{k'=1}^{10}\exp(\text{score}_{k'}(x_i))}
$$
for $k=1, \ldots, 10$.

To estimatethe model weights $W_{k, j}$ and $b_k$ for $k=1, \ldots, 10$ and $j=1, \ldots, 784$, the loss function is given by the negative log-likelihood (see also the section on gradient based method). The negative log-likelihood of a sample with input $x_i \in \mathbb R^{784}$ and label $y_i \in \{0, 1\}^{10}$, is given by the cross-entropy between the scores $p_k(x_i)$ and the label $y_i$:
$$
- \sum_{k=1}^{10} y_{i, k} \log(p_k(x_i))
$$
Stochastic gradient descent can be used to minimize this loss over small batches of data. 

In [ ]:
def softmax(x):
    m = x.max()
    e = np.exp(x - m)
    return e / e.sum()

In [ ]:
# Create a model prone to add layers sequentially
model = Sequential()
# flatten the data replaces 28 * 28 matrices by a 784 dimensional vector
# This is always necessary before a fully-connected layer (Dense object)
model.add(Flatten(input_shape=input_shape, name='flatten'))
# add one dense (fully connected layer) with softmax activation function
# As it is the first layer, the input size is mandatory
model.add(Dense(num_classes, activation='softmax', name='dense_softmax'))

# "compile" this model, 
model.compile(
    # specify the loss as the cross-entropy
    loss=keras.losses.categorical_crossentropy,
    # choose the gradient based method to estimate the parameters
    # see https://keras.io/optimizers/ to have an overview of the different options
    # see also section 2 on gradient based methods.
    optimizer=keras.optimizers.Adagrad(),
    # metric to monitor on the test data
    metrics=['accuracy']
)
model.summary()

In [ ]:
batch_size = 64
epochs = 10
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=1, label='Train')
plt.plot(history.epoch, history.history['val_acc'], lw=1, label='Test')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()

In [ ]:
weights, biases = model.get_layer('dense_softmax').get_weights()
imgs = weights.reshape(28, 28, 10)

fig = plt.figure(figsize=(10, 5))
vmin, vmax = imgs.min(), imgs.max()
for i in range(10):
    ax = plt.subplot(2, 5, i + 1)
    im = imgs[:, :, i]
    mappable = ax.imshow(im, interpolation="nearest", 
                         vmin=vmin, vmax=vmax, cmap='bwr')
    ax.axis('off')
    ax.set_title("%i" % i)
plt.tight_layout()

### <font color=darkred> Feed-Forward Neural Network (FFNN) </font>

The softmax regression of the previous section is a linear model, with 7850 parameters.  It is easy to fit, numerically stable, but might be too simple for some tasks.  The idea underlying neural networks is to have successive "neurons" performing a linear transformation of the input data (depending on a weight matrix and a bia vector) followed by an activation function to design more flexible models with additional parameters.

In [ ]:
# Create the graph for a fully connected feed-forward neural network with one hidden layer 
# with 128 units and a relu activation function. 

In [ ]:
model_ffnn = Sequential()

model_ffnn.add(Flatten(input_shape=input_shape))

model_ffnn.add(Dense(128, activation='relu'))

model_ffnn.add(Dense(num_classes, activation='softmax'))

How many parameters are involved in this model ?

The input size is 28*28 = 724. 

This input is transformed linearly in 128 hidden units in the dense layer which lead to 128*784 + 128 = 100480 parameters to obtain the 128 units.

These units are transformed in a ``softmax`` activation function which adds 10*128 + 10 = 1290 parameters.

This Feed Forward Neural Networks depends on 101770 parameters !

In [ ]:
model_ffnn.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adagrad(),
    metrics=['accuracy']
)

model_ffnn.summary()

In [ ]:
batch_size = 32
epochs = 10

# Run the train
history = model_ffnn.fit(x_train, y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         validation_data=(x_test, y_test))
score = model_ffnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=3, label='Training')
plt.plot(history.epoch, history.history['val_acc'], lw=3, label='Testing')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()

## <font color=darkred> Convolutional Neural Network </font>

In practice, increasing the size of hidden layers is not very effective. 
It is often a better idea to add more layers. 
Intuitively, if the observations have a hierarchical structure, adding more layers can be interpreted as a way to learn more levels of abstraction.  For example, in object detection tasks, it is easier to express shapes from edges and objects from shapes, than to express objects from pixels.  Thus, a good design should try to exploit this hierarchy.

In particular cases, such as grid-like data (time series, images), the observations can depend on a pattern which can be associated with different locations of the data. 
For example, an object can be in the middle or the left of the picture. 
Thus, the model has to be translation invariant: it is easier to learn how to recognize an object independently of its location. 

When two inputs might contain the same kind of information, then it is useful to share their weights and estimate the weights jointly for those inputs to learn statistical invariants (things that don't change much on average across time or space). 
Using this concept on images leads to convolutional neural networks (CNNs), on text, it results on recurrent neural networks (RNNs).  When using CNNs, weights are set to a small kernel that is used to perform a convolution across the image.

In the previous example, the step ``model_ffnn.add(Flatten(input_shape=input_shape))`` destroys the spatial organization of the input but is mandatory before the fully connected step: ``model_ffnn.add(Dense(128, activation='relu'))``. The spatial organization of the input may be preserved before using Flatten by convolution steps such as ``model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))``. This layer has 32 outputs, each of which depending on a 3*3 weight matrix.

Each output neuron of the next layer is computed as follows (image taken from https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/) before computing an elementwise RELU function:
<img src="conv.PNG">

The convolution maps patches of the input image, combined with the convolution kernel, for example
$$
\text{output} = \text{ReLU}(\text{patch} \times W + b)\,.
$$

Performing the convolution between the image and the weight matrix consists in moving the kernel across the image, and to produce an output for each patch. The way you move across the image is defined by two parameters:

- ``Stride``: the stride is the number of pixels you are shifting each time you move your kernel during the convolution.
- ``Padding``: defines what happens when the kernel reaches a border of the image when doing the convolution. 
"Valid" padding means that you stop at the edge, while "Same" padding allows to go off the edge and pad with zeros so that the width and the height of the output and input tensors are the same.

A classical approach is to use a stride of 1 and to combine theoutputs being in some neighborhood. Such an operation combining elements of a tensor is called ``pooling``: ``model_cnn.add(MaxPooling2D(pool_size=(2, 2)))``. Neighborhoods are define by the pooling window dimension (width x height) and the strides you use when moving this window across the image.  Again, a sliding window is applied along the input to produce the output, the most common function applied on each window being the ma function: it aggregates several outputs in a neighborhood $N$ using a max operation: 
$$
\text{output}'_i = \max_{j \in N}\text{output}_j, \quad i \in N.
$$
The formulas to compute the size of the ouput tensor are the same as for convolution padding and striding: (image taken from https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/):
<img src="maxpool.PNG">

In [ ]:
# Convolutional layer with 32 filters and 3 * 3 kernel sizes and 'relu' activation (use the `Conv2D` object)
# Convolutional layer with 64 filters and 3 * 3 kernel sizes and 'relu' activation (use the `Conv2D` object)
# Max pooling with pool size 2 * 2 (use the `MaxPooling2D` object)
# Dropout with probability 0.25 (use the `Dropout` object)
# Dense layer with 128 units with relu activation
# Dropout with probability 0.5
# Dense output layer with softmax activation

model_cnn = Sequential()

model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(num_classes, activation='softmax'))

model_cnn.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

model_cnn.summary()

In [ ]:
batch_size = 32
epochs = 5

# Run the train
history = model_cnn.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
score = model_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=3, label='Training')
plt.plot(history.epoch, history.history['val_acc'], lw=3, label='Testing')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()